This notebook is a demonstration of Apache Spark's potential use in the detection of Ethereum wallets which are being used for high volumes of anonymous and potentially illicit transactions. By using a modified version of Google's page rank algorithm, we can determine which Ethereum wallets are attracting an outsized volume of the blockchain's total transactions, and we can then cross-reference those wallets with the wallet ids of known legitimate businesses, removing those and leaving us with a list of suspicious wallet ids which need further AML (Anti-Money-Laundering) work needs to be done.

For a more technical explanation of how Google's page rank algorithm works than this notebook will provide, please refer here: https://en.wikipedia.org/wiki/PageRank 

**Warning, this notebook is designed to be run in Google Colab as it uses some built-in features to access Google Cloud Platform services like BigQuery data warehouses. This notebook can be made to work with other Jupyter notebook editors but it will require changes to how you access the BigQuery data warehouse.**

The block of code below downloads and installs Spark and Hadoop which we will need for this notebook. 

**Warning, do not change this.**

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

The block of code below sets up our environment and Spark application. 

**Warning, do not change this.**

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import random

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)
spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

The block of code below uses Google Colab's built-in authenticator to give the notebook access to the Google Cloud Platform services that are used later on, particularly the BigQuery Ethereum transaction data warehouse. If you authenticate successfully it will print "Authenticated".

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

The block of code below runs an SQL Query on Ethereum transaction data in a BigQuery data warehouse and saves it as a pandas data frame. For the sake of keeping the compute time and costs small I have put a limit of one hundred thousand rows however you are able to adjust the limit or remove it entirely and the rest of the code will work just fine. 

**Warning, if you run the query without a limit it will take a long time to process, the total size of the Ethereum blockchain is roughly 940GB**.

In [4]:
%%bigquery --project hardy-lightning-345701 df
SELECT from_address, to_address, value
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE value <> 0
LIMIT 100000

If you would like to see the data frame that was produced by our query, run the block of code below.

In [ ]:
print(df)

The block of code below takes the data frame that was produced by our query and converts it to a CSV file.

In [5]:
import csv
import pandas

df['from_address'].append(df['to_address']).shape
df.to_csv('eth.csv')

The block of code below takes the CSV file representing each individual transaction that we produced in the last block of code and converts it to a representation of a directed graph known as an adjacency list. In this representation, an Ethereum wallet id and its transactions are represented as a list of length 2 where the first item in the list is the id of the wallet sending Ethereum, and the second item is a list of pairs with each pair representing the wallet id that the original wallet has sent Ethereum to and the amount of Ethereum sent. This is an example of a wallet which has sent Ethereum to two other wallets: **[sending_wallet_id, [(recieving_wallet_id, value), (recieving_wallet_id, value)]]**

In [16]:
eth_df = spark.read.csv('eth.csv',sep=',',inferSchema=True, header=True)
eth_rdd = eth_df.rdd
eth_rdd_final = eth_rdd.map(lambda x: (x[1], (x[2], int(x[3])))).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], (list(x[1][i:i+2] for i in range(0, len(x[1]), 2))))).cache()
eth_rdd_final_count = eth_rdd_final.count()

The block of code below contains three functions init_rank, calculate_contribution, and modified_page_rank.

**init_rank:** This function allocates the initial page rank weight to each node in the transaction graph. The weight is allocated equally to each node.

**calculate_contribution:** This function calculates how much weight from the origin node to give to all the nodes which are connected to it. Unlike the basic page rank algorithm which distributes weight to each connected node equally, this function has been modified to give out a fraction of the weight based on the size of the transaction that was made to the other node. This fraction can be changed by adjusting the values of the num_nodes_fraction and value_fraction variables. A higher num_nodes_fraction will assign more weight to nodes with high transaction volume while a higher value_fraction will assign more weight to nodes with higher overall transaction value. If you are going to change these values make sure that they add up to 1.

**modified_page_rank:** This function iterates the page rank algorithm a set number of times and returns a set number of nodes with the highest page rank weight. The nodes are returned in the form of a list of pairs with each pair representing a wallet id and its page rank weight.

In [17]:
def init_rank(node):
  return node[0], 1/eth_rdd_final_count


def calculate_contribution(node):
  num_nodes = len(node[1][0])
  all_keys_count = eth_rdd_final_count
  sum_of_transactions = 0 
  num_nodes_fraction = 0.3
  value_fraction = 0.7
  for n in node[1][0]:
    sum_of_transactions = sum_of_transactions + n[1]
  lst = []
  if num_nodes == 0:
    return lst
  else:
    for n in node[1][0]:
      lst.append((node[0], ((node[1][1]*num_nodes_fraction)/num_nodes)+(node[1][1]*value_fraction)*(n[1]/sum_of_transactions)))
    return lst


def modified_page_rank(num_iterations, top_x):

  connected_transactions = eth_rdd_final
  rank = eth_rdd_final.map(lambda x: init_rank(x))

  for i in range(num_iterations):
    joined = connected_transactions.join(rank).flatMap(lambda x: calculate_contribution(x))
    rank = joined.reduceByKey(lambda x,y: x+y)

  sorted = rank.sortBy(lambda x: x[1], ascending=True)
  return sorted.take(top_x)

The modified page rank function produces a list of pairs of wallet addresses and their page rank weight in decreasing order. The block of code below prints the results of the modified page rank function in a more readable way, getting rid of the page rank values and printing only the wallet ids. These specific wallet ids can be looked up using a tool like https://etherscan.io/, and wallet ids without a public name attached to them that have a significant importance to the Ethereum blockchain are likely being used in ways that the people who control them do not want known about. 

**Warning, increasing the number of iterations of the modified_page_rank function results in diminishing returns to its accuracy as the page weights tend to converge after ~15 iterations, running it for more than that many times will likely result in wasted compute time for no real difference in the wallet ids returned.**

In [ ]:
for i in modified_page_rank(10, 20):
  print(i[0])